In [ ]:
import geopandas as gpd
import pandas as pd
import requests
import math
import numpy
from datetime import datetime
from shapely import wkt
from shapely.errors import WKTReadingError
from shapely.geometry import mapping, shape
from shapely import Polygon
import warnings
warnings.filterwarnings("ignore")
import re
import numpy as np

In [ ]:
login = {
    "username":"Dorito",
    "password":"Portador123"
}
# Crear una sesión para mantener la cookie
session = requests.Session()
response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)

In [ ]:
def get_data_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    datos = []
    while url:
        response = session.get(url, verify=False)
        info = response.json()
        datos.extend(info["results"])
        url = info["next"]
    return datos

def get_data_not_paginated(url):
    login = {
        "username":"Dorito",
        "password":"Portador123"
    }
    # Crear una sesión para mantener la cookie
    session = requests.Session()
    response = session.post("https://192.168.179.3/api/user/login/", login, verify=False)
    response = session.get(url, verify=False)
    info = response.json()
    return info


def parse_geom_safe(wkt_string):
    try:
        if isinstance(wkt_string, str) and "SRID=" in wkt_string:
            return wkt.loads(wkt_string.split(";", 1)[1])
    except:
        pass
    return None  # simplemente ignora lo inválido

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/Extensionista/"
print("cargando datos lotes")
datos = get_data_not_paginated(url)
df = pd.json_normalize(datos)


<h2>EXPORTANDO PRODUCTORES</h2>

In [ ]:
url_productor = "https://192.168.179.3/api/v1/productor/"
url_productor_2 = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"

print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_2 = get_data_not_paginated(url_productor_2)

df_productor = pd.json_normalize(datos_productor)
df_productor_2 = pd.json_normalize(datos_productor_2)

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df_productor.merge(
    df_productor_2,
    left_on="productor_data",
    right_on="_id",
    how="left"
)

<h2>CONSTRUYENDO LOS LOTES</h2>

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_lotes/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_productor_2 = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
url_fincas = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_lotes/mongo_get/mongo_atribute/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor")
datos_productor_2 = get_data_not_paginated(url_productor_2)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)
print("cargando datos lotes")
datos_lotes = get_data_not_paginated(url_lotes)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_2 = pd.json_normalize(datos_productor_2)
df_fincas = pd.json_normalize(datos_fincas)
df_lotes = pd.json_normalize(datos_lotes)


In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)
df_completo = df_completo.merge(
    df_productor_2,
    left_on="productor",
    right_on="postgres_data.id",
    how="left"
)

df_completo = df_completo.merge(
    df_fincas,
    left_on="finca",
    right_on="id",
    how="left"
)


df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)
print(df_completo.columns)
# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono_x"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")


In [ ]:
# columnas que sí quieres conservar
cols_keep = [
    'id_x', 'area_x', 'deshabilitar_poligono_x', 'documento','datos.nombre_completo', 'extensionista',
    'datos.numero_documento', 'datos.numero_lote', 'datos.fecha_visita',
    'datos.categoria', 'datos.subtipo_operacion', 'datos.fecha_actividad',
    'datos.variedad', 'datos.produccion', 'datos.estado_cultivo',
    'datos.distancia_surcos', 'datos.distancia_plantas', 'datos.densidad',
    'datos.numero_plantas', 'datos.gramos_plantas', 'datos.kg_produccion',
    'datos.descripcion', 'datos.observacion', 'geometry'
]

# filtrar el GeoDataFrame
gdf = gdf[cols_keep].drop_duplicates(subset=["id_x"])


In [ ]:
gdf.columns

In [ ]:
gdf.columns = ["id", "area", "deshabilitar_poligono", "documento", "nom_prod", "extensionista", "numero_documento", "numero_lote", "fecha_visita", 
"categoria", "subtipo_operacion", "fecha_actividad", "variedad", "produccion", "estado_cultivo", "distancia_surcos", "distancia_plantas", "densidad",
"numero_plantas", "gramos_plantas", "kg_produccion", "descripcion", "observacion", "geometry"
]

In [ ]:
import geopandas as gpd
import pandas as pd

def boolean_to_int(gdf: gpd.GeoDataFrame) -> gpd.GeoDataFrame:
    """
    Convierte todas las columnas booleanas de un GeoDataFrame
    a valores enteros (0 y 1).
    """
    gdf_copy = gdf.copy()
    for col in gdf_copy.select_dtypes(include=["bool"]).columns:
        gdf_copy[col] = gdf_copy[col].astype(int)
    return gdf_copy
gdf = boolean_to_int(gdf)

In [ ]:
gdf["documento"] = gdf["documento"].astype("string")
gdf["documento"] = (
    gdf["documento"]
    .astype(str)                # todo a string
    .str.replace(r"\.0$", "", regex=True)  # quita el .0 final
)


In [ ]:
gdf = gdf.sort_values(by='area', ascending=True).reset_index(drop=True)

In [ ]:
# df = pd.read_excel(r"C:\Users\dorito\Downloads\productores_en_area_protegida.xlsx", sheet_name='Hoja1')
# LISTA_CEDULA = df["cedula"].to_list()
# LISTA_CEDULA = [str(cedula) for cedula in LISTA_CEDULA]

In [ ]:
gdf["fecha_visita"] = pd.to_datetime(gdf["fecha_visita"], errors="coerce")
gdf["fecha_actividad"] = pd.to_datetime(gdf["fecha_actividad"], errors="coerce")
gdf["doc_prod"] = None
gdf["actualizar"] = 0
gdf["otro_asoc"] = 0
gdf["calcular_p"] = 0

In [ ]:
gdf = gdf.drop_duplicates(subset=["geometry"])

In [ ]:
gdf.to_file("../Lote g.shp")

<H2>ARREGLANDO LAS FINCAS</H2>

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_fincas/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_productor_2 = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
url_lotes = "https://192.168.179.3/api/v1/poligonos_fincas/mongo_get/mongo_atribute/"
print("cargando datos fincas")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos productor_2")
datos_productor_2 = get_data_not_paginated(url_productor_2)
print("cargando datos fincas atributos")
datos_lotes = get_data_not_paginated(url_lotes)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_productor_2 = pd.json_normalize(datos_productor_2)
df_lotes = pd.json_normalize(datos_lotes)

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)
df_completo = df_completo.merge(
    df_productor_2,
    left_on="productor",
    right_on="postgres_data.id",
    how="left"
)
df_completo = df_completo.merge(
    df_lotes,
    left_on="id_x",
    right_on="poligono_id",
    how="left"
)
# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)
print(df_completo.columns)
# Crear el GeoDataFrame sin que explote
gdf_fincas = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
# columnas que sí quieres conservar
cols_keep = ['id_x', 'area', 'deshabilitar_poligono', 'codigo_finca',
'documento', 'extensionista', 'datos.nombre_completo','datos.nombre_finca',
'datos.categoria', 'datos.fecha_visita','datos.descripcion', 'datos.observaciones', 'geometry']

# filtrar el GeoDataFrame
gdf_fincas = gdf_fincas[cols_keep].drop_duplicates(subset=["id_x"])

In [ ]:
gdf_fincas.columns

In [ ]:
gdf_fincas.columns = ['id', 'area', 'deshabilitar_poligono', 'codigo_finca',
'documento', 'extensionista', 'nom_prod', 'nombre_finca',
'categoria', 'fecha_visita','descripcion', 'observaciones', 'geometry']

In [ ]:
gdf_fincas = gdf_fincas.sort_values(by='area', ascending=True).reset_index(drop=True)

In [ ]:
gdf_fincas["fecha_visita"] = pd.to_datetime(gdf_fincas["fecha_visita"], errors="coerce")
gdf_fincas["doc_prod"] = None
gdf_fincas["actualizar"] = 0
gdf_fincas["otro_asoc"] = 0
gdf_fincas["calcular_p"] = 0


In [ ]:
gdf_fincas["documento"] = gdf_fincas["documento"].astype("string")
gdf_fincas["documento"] = (
    gdf_fincas["documento"]
    .astype(str)                # todo a string
    .str.replace(r"\.0$", "", regex=True)  # quita el .0 final
)


In [ ]:
gdf_fincas["datos.fecha_visita"] = pd.to_datetime(
    gdf["datos.fecha_visita"],
    format="%Y-%m-%d",
    errors="coerce"
)


In [ ]:
gdf_fincas.to_file("../Finca.shp")

In [ ]:
jhkjhkjkj

<H3>POLIGONOS CONSERVACION</H3>

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_conservacion/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_fincas = "https://192.168.179.3/api/v1/poligonos_conservacion/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_fincas = pd.json_normalize(datos_fincas)
# df_lotes = pd.json_normalize(datos_lotes)

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

# df_completo = df_completo.merge(
#     df_lotes,
#     left_on="id_x",
#     right_on="poligono_id",
#     how="left"
# )

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf.to_file("./conservacion.shp")

<h3>Poligonos Infraestructura</h3>

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/poligonos_infraestructura/"
url_productor = "https://192.168.179.3/api/v1/productor/"
url_fincas = "https://192.168.179.3/api/v1/poligonos_infraestructura/"
print("cargando datos lotes")
datos = get_data_paginated(url)
print("cargando datos productor")
datos_productor = get_data_not_paginated(url_productor)
print("cargando datos fincas")
datos_fincas = get_data_paginated(url_fincas)

df = pd.json_normalize(datos)
df_productor = pd.json_normalize(datos_productor)
df_fincas = pd.json_normalize(datos_fincas)
# df_lotes = pd.json_normalize(datos_lotes)

In [ ]:
# Filtrar productores que tienen postgres_data.id
df_productor = df_productor[df_productor["id"].notna()]
# Realizar los merge sin eliminar filas del DataFrame principal
df_completo = df.merge(
    df_productor,
    left_on="productor",
    right_on="id",
    how="left"
)

# df_completo = df_completo.merge(
#     df_lotes,
#     left_on="id_x",
#     right_on="poligono_id",
#     how="left"
# )

# Aplicar solo a las válidas
df_completo["geometry"] = df_completo["poligono"].apply(parse_geom_safe)

# Crear el GeoDataFrame sin que explote
gdf = gpd.GeoDataFrame(df_completo, geometry="geometry", crs="EPSG:4326")

In [ ]:
gdf.to_file("./Infraestructura.shp")

<h3>Arreglando errores duplicados y asi</h3>

In [ ]:
# import geopandas as gpd

# def detectar_invalidos(gdf):
#     # Caso 1: geometrías None o vacías
#     vacios = gdf[gdf["geometry"].is_empty | gdf["geometry"].isna()]

#     # Caso 2: geometrías inválidas
#     invalidos = gdf[~gdf["geometry"].is_valid]

#     # Unir ambos
#     resultado = pd.concat([vacios, invalidos]).drop_duplicates()

#     return resultado

# # Ejemplo de uso
# gdf_invalidos = detectar_invalidos(gdf)
# print("Cantidad de geometrías vacías o inválidas:", len(gdf_invalidos))

# # Si quieres exportar para revisar en QGIS
# if not gdf_invalidos.empty:
#     gdf_invalidos.to_file("Poligonos_invalidos.shp")


In [ ]:
# import geopandas as gpd

# def detectar_solapados(gdf, umbral=0.8):
#     # aseguramos que todo está en la misma proyección métrica para calcular áreas
#     if gdf.crs.is_geographic:
#         gdf = gdf.to_crs(3857)  

#     overlaps = []

#     for i, geom1 in gdf.iterrows():
#         print(f"{i}/{len(gdf)}")
#         for j, geom2 in gdf.iterrows():
#             if i >= j:  # evitar comparar dos veces o consigo mismo
#                 continue
#             inter = geom1.geometry.intersection(geom2.geometry)

#             if not inter.is_empty:
#                 area_inter = inter.area
#                 area1 = geom1.geometry.area
#                 area2 = geom2.geometry.area

#                 # porcentaje relativo de intersección
#                 porc1 = area_inter / area1
#                 porc2 = area_inter / area2

#                 if porc1 >= umbral or porc2 >= umbral:
#                     overlaps.append({
#                         "id1": i,
#                         "id2": j,
#                         "area_inter": area_inter,
#                         "porc1": porc1,
#                         "porc2": porc2
#                     })

#     return gpd.GeoDataFrame(overlaps)

# # Ejemplo de uso
# result = detectar_solapados(gdf, umbral=0.8)

# print(result)
# result.to_file("Lotes_a_revisar.shp")


In [ ]:

# # sacar los IDs únicos que participan en solapamientos
# ids_solapados = set(result["id1"]).union(set(result["id2"]))

# # filtrar el gdf original para quedarnos con solo los que se solapan
# gdf_solapados = gdf.loc[gdf.index.isin(ids_solapados)]

# print(f"Se detectaron {len(ids_solapados)} polígonos solapados")
# gdf_solapados.to_file("Lotes_solapados.shp")

In [ ]:
# gdf

In [ ]:
# gdf["fecha"] = pd.to_datetime(
#     gdf["fecha"], 
#     format="%d/%m/%Y %H:%M:%S", 
#     errors="coerce"   # pone NaT si hay errores de formato
# )

In [ ]:
# gdf_finca

In [ ]:
# gdf_solapados_filtro.columns

In [ ]:
# ids_analizados = []
# ids_eliminar = []

# for _, row in result.iterrows():  # recorremos filas del DataFrame result
#     print(f"{_}/{len(result)}")
#     id_1 = row["id1"]
#     id_2 = row["id2"]

#     # si ya procesamos alguno de los dos, saltamos
#     if id_1 in ids_analizados or id_2 in ids_analizados:
#         continue  

#     ids_analizados.append(id_1)
#     ids_analizados.append(id_2)
#     gdf_solapados_filtro = gdf.loc[gdf.index.isin([id_1, id_2])]
#     union_gdf1 = gdf_solapados_filtro.unary_union    
#     gdf_fincas_intersectadas = gdf_finca[gdf_finca.intersects(union_gdf1)]

#     if not gdf_fincas_intersectadas.empty:
#         lista_documentos = gdf_fincas_intersectadas["documento"].to_list()
#         gdf_solapados_filtro_exorcismo = gdf_solapados_filtro.loc[~gdf_solapados_filtro["documento"].isin(lista_documentos)]
#         if not gdf_solapados_filtro_exorcismo.empty:
#             for idx in gdf_solapados_filtro_exorcismo["id_x"]:
#                 ids_eliminar.append(idx)
#             continue


#     gdf_mas_antiguo = gdf_solapados_filtro.loc[[gdf_solapados_filtro["fecha"].idxmin()]]
#     if not gdf_mas_antiguo.empty:
#         ids_eliminar.append(gdf_mas_antiguo["id_x"].iloc[0])

In [ ]:
# for i, id_eliminar in enumerate(ids_eliminar):
#     print(f"{i}/{len(ids_eliminar)}")
#     url = f"https://192.168.179.3/api/v1/poligonos_lotes/{id_eliminar}/"
#     print(url)
#     session.delete(url, verify=False)

In [ ]:
# gdf = gpd.read_file(r"C:\Users\dorito\Downloads\trazabilidad.geojson")
# gdf

In [ ]:
# # tomar el centroide del polígono
# gdf["lon"] = gdf.geometry.centroid.x
# gdf["lat"] = gdf.geometry.centroid.y

In [ ]:
# gdf.to_file("./trazabilidad.geojson")

In [ ]:
# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/productor/"
print("cargando datos lotes")
datos = get_data_not_paginated(url)
df = pd.json_normalize(datos)

# Obtener y normalizar datos
url = "https://192.168.179.3/api/v1/productor/mongo_get/productor_data/"
print("cargando datos lotes")
datos = get_data_not_paginated(url)
df_productor = pd.json_normalize(datos)


In [ ]:
df["productor_data"] = df["productor_data"].astype(str)
df_productor["postgres_data.id"] = df_productor["postgres_data.id"].astype(str)

In [ ]:
df_filtrado["productor_data"]

In [ ]:
df_filtrado = df[df["productor_data"].notna()].copy()
df_total = df_filtrado.merge(
    df_productor,
    left_on="id",
    right_on="postgres_data.id",
    how="left"
)



In [ ]:
df_total["datos.fecha_afiliacion"].str.startswith("2025")

In [ ]:
df_2025 = df_total[df_total["datos.fecha_afiliacion"].str.startswith("2025", na=False)].copy()

print(f"Filas originales: {len(df_total)}")
print(f"Filas del año 2025: {len(df_2025)}")


In [ ]:
df_2025

In [ ]:
import unicodedata

def quitar_tildes(texto):
    """Elimina tildes y caracteres especiales de un texto."""
    if not isinstance(texto, str):
        return ""
    return "".join(
        c for c in unicodedata.normalize("NFD", texto)
        if unicodedata.category(c) != "Mn"
    )
j = 1
for i, row in df_2025.iterrows():
    print(f"{j}/{len(df_2025)}")
    j += 1
    if pd.isna(row["datos.nombre_completo"]):
        print(f"⚠️ No se puede trabajar con {row['documento']}")
        continue
    if pd.isna(row["documento"]):
        print(f"⚠️ No tiene documento {row['id']}")
        continue

    codigo_inicio = "COOPR2025"

    # --- Procesar nombre ---
    nombre = quitar_tildes(row["datos.nombre_completo"]).upper()
    iniciales = "".join([p[0] for p in nombre.split() if p])
    print(row["documento"])
    # --- Procesar documento ---
    documento = str(int(row["documento"]))  # Por si viene como float o con .0
    ultimos_3 = documento[-3:] if len(documento) >= 3 else documento.zfill(3)

    # --- Crear código final ---
    codigo = f"{codigo_inicio}{iniciales}{ultimos_3}"
    objeto = {
        "codigo_productor": codigo
    }

    # df_actualizar.at[i, "codigo_generado"] = codigo
    # print(f"{nombre} ({documento}) → {codigo}")
    session.patch(f"https://192.168.179.3/api/v1/productor/{row['id']}", json=objeto, verify=False)

In [ ]:
df = pd.read_excel("../temp.xlsx")

In [ ]:
for i, row in df.iterrows():
    url = f"https://192.168.179.3/api/v1/productor/sql/?documento={row['CEDULA']}"
    respuesta = session.get(url, verify=False)
    respuesta = respuesta.json()[0]
    df.at[i, "id"] = respuesta["id"]
    df.at[i, "codigo_productor"] = respuesta["codigo_productor"]

In [ ]:
import requests

for i, row in df.iterrows():
    url = f"https://192.168.179.3/api/v1/productor/{row['id']}/mongo_get/productor_data/"
    print(f"\n🔗 URL: {url}")

    respuesta = {"productor_data":{
        "activo": True,
        "delegado": False,
        "nombre_completo": row["NOMBRE "],
        "fecha_afiliacion": "2025-09-01"
        }
    }

    print("📤 Enviando datos:", respuesta)

    try:
        r = session.post(url, json=respuesta, verify=False)
        print("📥 Estado:", r.status_code)
        
        # Mostrar el cuerpo de la respuesta
        try:
            print("📦 Respuesta JSON:", r.json())
        except ValueError:
            print("📄 Respuesta texto:", r.text)

    except requests.exceptions.RequestException as e:
        print("❌ Error de conexión:", e)


In [ ]:
folder_fotos = r"C:\Users\dorito\Downloads"
puntos_fotos = gpd.read_file(r"C:\Users\dorito\Documents\Posicion Fotos Lotes.shp")


In [ ]:
puntos_fotos["longitud"] = puntos_fotos.geometry.x
puntos_fotos["latitud"] = puntos_fotos.geometry.y

In [ ]:
puntos_fotos.columns

In [ ]:
import os
puntos_fotos["ruta"] = puntos_fotos['Nombre Lot'].apply(lambda x: os.path.join(folder_fotos, f"{x}.jpg"))

<h2>ORGANIZANDO INFORMACION A IBARRA RELACION DE ASOCIADOS PROBLEMAS EN LA BASE</h2>

In [ ]:
asociados_a_revisar = [4883329,4917242,12229437,12295051,14214331,79136928,79749725,83239785,83250337]

In [ ]:
gdf.columns

In [ ]:
gdf_revisar = gdf[["id_x","documento", "datos.numero_lote", "datos.numero_plantas"]].loc[gdf["documento"].isin(asociados_a_revisar)]

In [ ]:
gdf_revisar.loc[(gdf_revisar["datos.numero_plantas"].isna())|(gdf_revisar["datos.numero_plantas"] == 0)]

In [ ]:
df = pd.read_excel(r"C:\Users\dorito\Downloads\POLIGONOS COCENTRAL AULA MAYER VERIFICADOS.xlsx")
gdf_1 = gpd.read_file("./POLIGONOS TRAZABILIDAD 17 10 2025.xlsx/PRIMERA PARTE FALTANTE.geojson")
gdf_2 = gpd.read_file("./POLIGONOS TRAZABILIDAD 17 10 2025.xlsx/SEGUNDA PARTE FALTANTE.geojson")

In [ ]:
gdf_2 = gdf_2.rename(columns={"CEDULA": "documento"})
gdf_2.columns


In [ ]:
df.columns

In [ ]:
gdf_total = gpd.GeoDataFrame(pd.concat([gdf_1,gdf_2]), crs=4326, geometry="geometry")

In [ ]:
from shapely import unary_union
gdfs = []
for documento in gdf_total["documento"].unique():
    documento = int(documento)
    df_filtro = df.loc[df["DOCUMENTO"] == documento]
    gdf_filtro = gdf_total.loc[gdf["documento"] == str(documento)]
    df_filtro["geometry"] = unary_union(gdf_filtro.geometry)
    gdfs.append(df_filtro)
gdf_traza = gpd.GeoDataFrame(pd.concat(gdfs), geometry="geometry", crs=4326)

In [ ]:
gdf_traza = gdf_traza.drop(
    columns=[col for col in gdf_traza.columns if "unnamed" in col.lower()]
)
gdf_traza

In [ ]:
gdf_traza.to_file("./Anexo_Poligonos_Laumayer.geojson")